In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [3]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

In [4]:
from tensorflow.keras.layers import GRU

#### 導入資料

In [5]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

In [6]:
x_train_ = sequence.pad_sequences(x_train, maxlen=100)
x_test_ = sequence.pad_sequences(x_test, maxlen=100)

# 建立神經網路-LSTM

In [7]:
model = Sequential()

In [8]:
model.add(Embedding(10000,128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [9]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         1280000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,411,713
Trainable params: 1,411,713
Non-trainable params: 0
_________________________________________________________________


### 參數數計算：
### 一個 LSTM的神經元有 f閘  , i閘 , o閘 ,和一個用於更新cell state的Ｃ‘ ，然後輸出 C 和 hidden_state
* f閘: 前一個hidden_state \* 權重 ＋ 輸入 \* 權重2 + 偏值 ，然後再作用於 sigmoid ，共（輸入數＋LSTM神經元數＋1）個參數
* i閘: 前一個hidden_state \* 權重 ＋ 輸入 \* 權重2 + 偏值 ，然後再作用於 sigmoid ，共（輸入數＋LSTM神經元數＋1）個參數
* z閘: 前一個hidden_state \* 權重 ＋ 輸入 \* 權重2 + 偏值 ，然後再作用於 sigmoid ，共（輸入數＋LSTM神經元數＋1）個參數
* Ｃ’: 前一個hidden_state \* 權重 ＋ 輸入 \* 權重2 + 偏值 ，然後再作用於 tanh    ，共（輸入數＋LSTM神經元數＋1）個參數
* Ｃ ：f閘 \* 前一個Ｃ ＋ i閘 \* Ｃ ，共零個參數
* hidden_state： o閘 \* tanh( Ｃ‘） ，共零個參數
### 輸入是128,  LSTM的神經元數是128 ，於是一個LSTM有（128+128+1)*4個參數
### 所以這一層有（１２８＋１２８＋１）* 4 * 128個參數

In [21]:
(128+128+1)*4*128

131584

#### 訓練

In [11]:
model.fit(x_train_ ,y_train, batch_size=32, epochs=10, validation_data=(x_test_,y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 273s 11ms/sample - loss: 0.4593 - accuracy: 0.7866 - val_loss: 0.3753 - val_accuracy: 0.8357
Epoch 2/10
25000/25000 [==============================] - 273s 11ms/sample - loss: 0.3263 - accuracy: 0.8628 - val_loss: 0.3611 - val_accuracy: 0.8424
Epoch 3/10
25000/25000 [==============================] - 269s 11ms/sample - loss: 0.2654 - accuracy: 0.8931 - val_loss: 0.3773 - val_accuracy: 0.8340
Epoch 4/10
25000/25000 [==============================] - 264s 11ms/sample - loss: 0.2074 - accuracy: 0.9200 - val_loss: 0.4416 - val_accuracy: 0.8427
Epoch 5/10
25000/25000 [==============================] - 264s 11ms/sample - loss: 0.1677 - accuracy: 0.9371 - val_loss: 0.4419 - val_accuracy: 0.8288
Epoch 6/10
25000/25000 [==============================] - 265s 11ms/sample - loss: 0.1268 - accuracy: 0.9538 - val_loss: 0.5619 - val_accuracy: 0.8315
Epoch 7/10
25000/25000 [====================

#### 查看準確率

In [13]:
model.evaluate(x_test_, y_test, verbose=0)

[0.7310195725655556, 0.8346]

* 準確率為 83.46%

# 建立神經網路-GRU

In [13]:
model2 = Sequential()

In [14]:
model2.add(Embedding(10000,128))
model2.add(GRU(128, dropout=0.2, recurrent_dropout=0.2))
model2.add(Dense(1, activation='sigmoid'))

In [15]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         1280000   
_________________________________________________________________
gru (GRU)                    (None, 128)               99072     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,379,201
Trainable params: 1,379,201
Non-trainable params: 0
_________________________________________________________________


### 參數數計算
### 一個GRU的神經元有 r閘, z閘, 和一個更新用的S' ，然後輸出Ｓ
* r閘：前一個Ｓ \* 權重 ＋ 輸入 \* 權重2 ＋ 偏值 ，然後作用於 sigmoid ，共（輸入數 ＋ GRU神經元數 ＋ 1 ）個參數
* z閘：前一個Ｓ \* 權重 ＋ 輸入 \* 權重2 ＋ 偏值 ，然後作用於 sigmoid ，共（輸入數 ＋ GRU神經元數 ＋ 1 ）個參數
* S' ：前一個Ｓ \* r閘 \*權重  + 輸入 \* 權重2 ＋ 偏值 ，然後作用於 tanh    ，共（輸入數 ＋ GRU神經元數 ＋ 1 ）個參數
* S  ：（1-z閘）\* 前一個S ＋ z閘 \* S' ， 零個參數

### 輸入數是 128 ， GRU神經元數是128， 共 （(128+128+1)\*3*128 個參數

In [17]:
(128+128+1)*3*128 

98688

### 然而計算結果不符合預期，實際上是(128+128+2)\*3*128 
猜測：應該是 輸入 和 前一個Ｓ都各有一個偏值

#### 訓練

In [19]:
model2.fit(x_train_, y_train)

Train on 25000 samples
25000/25000 [==============================] - 219s 9ms/sample - loss: 9.9332 - accuracy: 0.6767


#### 查看準確率

In [22]:
model2.evaluate(x_test_,y_test,verbose=0)

[0.567109863204956, 0.7]

* 準確率為:70%

### 看起來LSTM在這個模型好像要準確一些
### 為了讓model公平（？）一些，調整GRU的神經元數，使其參數數約等於LSTM的

In [25]:
model2 = Sequential()

In [26]:
model2.add(Embedding(10000,128))
model2.add(GRU(154, dropout=0.2, recurrent_dropout=0.2))
model2.add(Dense(1, activation='sigmoid'))

In [27]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         1280000   
_________________________________________________________________
gru_1 (GRU)                  (None, 154)               131208    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 155       
Total params: 1,411,363
Trainable params: 1,411,363
Non-trainable params: 0
_________________________________________________________________


#### 參數數計算

In [30]:
(154+128+2)*3*154

131208

#### 訓練

In [31]:
model2.fit(x_train_, y_train)

Train on 25000 samples
25000/25000 [==============================] - 236s 9ms/sample - loss: 23.2446 - accuracy: 0.6790


#### 查看準確率

In [33]:
model2.evaluate(x_test_,y_test,verbose=0)

[0.5779512642192841, 0.68708]

* 準確率為：68.71%

# 結論：LSTM在這個模型好像真的要準確一些